# iCaRL: Incremental Classifier and Representation Learning

In [ ]:
%load_ext autoreload
%autoreload 2

# Libraries

In [ ]:
import pickle
import pandas as pd
import seaborn as sns
from tqdm import tqdm
from torchvision.transforms import Compose
from torchvision.transforms.v2 import Identity

In [ ]:
from avalanche.benchmarks.classic import PermutedMNIST
from avalanche.models import make_icarl_net, initialize_icarl_net
from avalanche.training import ICaRL
from avalanche.training.plugins import EvaluationPlugin
from avalanche.logging import InteractiveLogger
from avalanche.evaluation.metrics import (
    accuracy_metrics
)

from torch.nn import CrossEntropyLoss
from torch.optim import SGD

## Custom Libraries

In [ ]:
import sys
sys.path.append("../base_code/")

from base_code.constants import DATASETS_PATH, SAVED_METRICS_PATH

# Dataset and definitions

In [ ]:
buffer_transform = Compose([
    Identity(),
])

## Dataset loading

We load state-of-the-art dataset Modified NIST

In [ ]:
scenario = PermutedMNIST(10, seed=1234, dataset_root=DATASETS_PATH)

## Scenario creation with train test streaming

In this point, we define our scenario considering a training where in every experience of it, a new class is presented. This is, first we train with a class $a$, the following experience we train with class $b$ ($a \neq b$)

In [ ]:
train_stream = scenario.train_stream
test_stream = scenario.test_stream

## Evaluation metrics definition

In [ ]:
eval_plugin = EvaluationPlugin(
    accuracy_metrics(experience=True, stream=True),
    loggers=[InteractiveLogger()]
)

## Model, Optimizer, Loss function and Strategy definition

* `model`: Multi Layer Perceptron
* `Optimizer`: Adam
* `Loss function`: Cross Entropy
* `Strategy`: Elastic Weight Consolidation

In [ ]:
model = make_icarl_net(num_classes=10, c=1)
model.apply(initialize_icarl_net)
optimizer = SGD(model.parameters(), lr=1e-3)
strategy = ICaRL(
    model.feature_extractor, model.classifier, optimizer,
    buffer_transform=buffer_transform,
    memory_size=1000,
    fixed_memory=True,
    train_epochs=2,
    train_mb_size=32, eval_mb_size=32,
    evaluator=eval_plugin
)

# Training and evaluation

Revisar porque el entrenamiento se está comportando de forma rara

In [ ]:
results = list()

for (task_id, experience) in tqdm(enumerate(train_stream, start=1)):
    strategy.train(experience)
    metrics = strategy.eval(test_stream[:task_id])
    results.append(metrics)

In [ ]:
accuracies: dict[int, list[float]] = dict()

for i in range(10):
    accuracies[f"Task{i}"] = [None]*i + eval_plugin.get_all_metrics()[f"Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp00{i}"][1]

accuracies["Overall"] = eval_plugin.get_all_metrics()["Top1_Acc_Stream/eval_phase/test_stream/Task000"][1]

In [ ]:
eval_plugin.get_all_metrics()

In [ ]:
acc_df = pd.DataFrame(accuracies)
acc_df.index = range(10)

In [ ]:
# lineplot every task acc per training
sns.lineplot(data=acc_df, dashes=False, markers=True)

# Qué clase se entrenó en cada tarea / experiencia?

# Store metrics

In [ ]:
pickle.dump(acc_df, open(SAVED_METRICS_PATH / "ewc.pkl", "wb"))